<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div>

> **YugabyteDB YSQL Development**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/).
>

# Language fundamentals
This notebook showcases various Data Definition Language (DDL) and Data Manipulation Language (DML) statements for Yugabyte Structure Query Language (YSQL).

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Connect to the `ds_ybu` database, *which you must do next*
- ☑️ Run through a series of DDL and DML scenarios
  -  Basic of DDL and DML
  -  Built-in Functions
  -  Advanced Features

---
## 👣 Setup steps
Here are the steps to setup this lab:
- Import the notebook variables
- Connect to `db_ybu` database
- Load the SQL Magic extension for the connection
- Create the prepared statements

### Import the notebook variables 

> ‼️ Important
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.

In [1]:
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
# Use %store -r to read 01_Lab_Setup variables
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER

%store -r MY_DATA_DDL_FILE_1
%store -r MY_DATA_DML_FILE_1
%store -r MY_DATA_DDL_FILE_2
%store -r MY_DATA_DML_FILE_2
%store -r MY_DATA_DDL_FILE_3
%store -r MY_DATA_DML_FILE_3

## Connect to the `db_ybu` database
Run all the cells in this section:
- Connect using Python and PostgreSQL driver
- Load the SQL magic extension
- Create the prepared statements


In [2]:
# connect use Python 3.7.9+
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine


db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql://yugabyte@'+db_host+':5433/'+db_name

engine = create_engine(connection_str)

### Load the SQL magic extension

In [3]:
%reload_ext sql

# SQL magic for python connection string
%sql {connection_str}

### Create the prepared statements

> IMPORTANT!
>   
> In order to create the prepared statements for the SQL magic connection, you must run the following cell!!!
> 
> Do not skip this step.
> 

In [4]:
#%% python, but prepared statements as sql magic
if (MY_GITPOD_WORKSPACE_URL is None):
    a = %sql select fn_yb_create_stmts()
else:
    WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
    a = %sql select fn_yb_create_stmts(:WORKSPACE_URL)

print (a)

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
+----------------------------------+
|        fn_yb_create_stmts        |
+----------------------------------+
| 2023-04-20 16:23:53.147778-07:00 |
+----------------------------------+


Confirm that the following query returns a count of 3 (for three prepared statements).

In [5]:
%%sql 
select count(*) from pg_prepared_statements where 1=1 and name in ('stmt_util_metrics_snap_tablet','stmt_util_metrics_snap_table','stmt_util_metrics_snap_reset')

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


count
3


---


## q1 | Basics of Data Definition Language (DDL) and Data Manipulation Language (DML)

<!--

insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7369, 'SMITH', 'CLERK', 7902, '1980-12-17', 800, NULL, 20,'SMITH@acme.com', '{"skills":["accounting"]}'), (7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600, 300, 30,'ALLEN@acme.com', null), (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250, 500, 30,'WARD@compuserve.com', null), (7566, 'JONES', 'MANAGER', 7839, '1981-04-02', 2975, NULL, 20,'JONES@gmail.com', null), (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250, 1400, 30,'MARTIN@acme.com', null), (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850, NULL, 30,'BLAKE@hotmail.com', null), (7782, 'CLARK', 'MANAGER', 7839, '1981-06-09', 2450, NULL, 10,'CLARK@acme.com', '{"skills":["C","C++","SQL"]}'), (7788, 'SCOTT', 'ANALYST', 7566, '1982-12-09', 3000, NULL, 20,'SCOTT@acme.com', '{"cat":"tiger"}'), (7839, 'KING', 'PRESIDENT', NULL, '1981-11-17', 5000, NULL, 10,'KING@aol.com', null), (7844, 'TURNER', 'SALESMAN', 7698, '1981-09-08', 1500, 0, 30,'TURNER@acme.com', null), (7876, 'ADAMS', 'CLERK', 7788, '1983-01-12', 1100, NULL, 20,'ADAMS@acme.org', null), (7900, 'JAMES', 'CLERK', 7698, '1981-12-03', 950, NULL, 30,'JAMES@acme.org', null), (7902, 'FORD', 'ANALYST', 7566, '1981-12-03', 3000, NULL, 20,'FORD@acme.com', '{"skills":["YSQL","YCQL","SQL","CQL"]}'), (7934, 'MILLER', 'CLERK', 7782, '1982-01-23', 1300, NULL, 10,'MILLER@acme.com', null);
>



### Scenario 1: Create a table and insert rows

Create two related tables, `tbl_departments` and `tbl_employees`:

In [6]:
%%sql

drop table if exists tbl_departments;

create table if not exists tbl_departments (
  deptno integer NOT NULL,
  dname text,
  loc text,
  description text,
  CONSTRAINT pk_dept PRIMARY KEY (deptno asc)
);

drop table if exists tbl_employees;

create table if not exists tbl_employees (
  empno integer generated by default as identity (start with 10000) NOT NULL,
  ename text NOT NULL,
  job text,
  mgr integer,
  hiredate date,
  sal integer,
  comm integer,
  deptno integer NOT NULL,
  email text,
  other_info jsonb,
  CONSTRAINT pk_emp PRIMARY KEY (empno hash),
  CONSTRAINT emp_email_uk UNIQUE (email),
  CONSTRAINT emp_email_check CHECK ((email ~ '^[a-zA-Z0-9.!#$%&''*+/=?^_`{|}~-]+@[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(?:\.[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?)*$'::text))
);


 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.
Done.
Done.
Done.


[]

Insert data into `tbl_departments`:

In [7]:
%%sql
insert into tbl_departments (deptno, dname, loc, description)
values (10, 'ACCOUNTING', 'NEW YORK','preparation of financial statements, maintenance of general ledger, payment of bills, preparation of customer bills, payroll, and more.'), (20, 'RESEARCH', 'DALLAS','responsible for preparing the substance of a research report or security recommendation.'), (30, 'SALES', 'CHICAGO','division of a business that is responsible for selling products or services'), (40, 'OPERATIONS', 'BOSTON','administration of business practices to create the highest level of efficiency possible within an organization');

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
4 rows affected.


[]

Insert data into `tbl_employees`:

In [8]:
%%sql

insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7369, 'SMITH', 'CLERK', 7902, '1980-12-17', 800, NULL, 20,'SMITH@acme.com', '{"skills":["accounting"]}');
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600, 300, 30,'ALLEN@acme.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250, 500, 30,'WARD@compuserve.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7566, 'JONES', 'MANAGER', 7839, '1981-04-02', 2975, NULL, 20,'JONES@gmail.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250, 1400, 30,'MARTIN@acme.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850, NULL, 30,'BLAKE@hotmail.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7782, 'CLARK', 'MANAGER', 7839, '1981-06-09', 2450, NULL, 10,'CLARK@acme.com', '{"skills": ["C","C++","SQL"] }');
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7788, 'SCOTT', 'ANALYST', 7566, '1982-12-09', 3000, NULL, 20,'SCOTT@acme.com', '{"cat":"tiger"}');
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES  (7839, 'KING', 'PRESIDENT', NULL, '1981-11-17', 5000, NULL, 10,'KING@aol.com', null); 
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7844, 'TURNER', 'SALESMAN', 7698, '1981-09-08', 1500, 0, 30,'TURNER@acme.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7876, 'ADAMS', 'CLERK', 7788, '1983-01-12', 1100, NULL, 20,'ADAMS@acme.org', null); 
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7900, 'JAMES', 'CLERK', 7698, '1981-12-03', 950, NULL, 30,'JAMES@acme.org', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7902, 'FORD', 'DEVELOPER', 7566, '1981-12-03', 3000, NULL, 20,'FORD@acme.com', '{"skills":["YSQL","YCQL","SQL","CQL"]}'); 
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7934, 'MILLER', 'CLERK', 7782, '1982-01-23', 1300, NULL, 10,'MILLER@acme.com', null);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]


Learn to use YSQL to update, access, and analyze the data stored in a database.

### Scenario 2: SQL Updates

Update salary for all employees who are not managers

Description:
A SQL update can compute the new value and return it without the need to query again. The following adds 100 to the salaries of all employees who are not managers and show the new value

In [9]:
%%sql /* row counts */

update tbl_employees set sal=sal+100
where job != 'MANAGER'
returning ename,sal as new_salary;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
11 rows affected.


ename,new_salary
SMITH,900
ADAMS,1200
WARD,1350
KING,5100
FORD,3100
MARTIN,1350
JAMES,1050
ALLEN,1700
MILLER,1400
SCOTT,3100


### Scenario 3: Join

List all employees earning more than their managers using a self-join query.

Description:
A self join is a regular join, but the table is joined with itself. The following SQL statement matches employees with their manager and filters those that are earning more than their manager.

In [10]:
%%sql

select 
  employee.ename,
  employee.sal,
  manager.ename as "manager ename",
  manager.sal as "manager sal"
from
  tbl_employees employee
join tbl_employees manager ON
  employee.mgr = manager.empno
where 1=1 
 and manager.sal<employee.sal
order by employee.sal;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


ename,sal,manager ename,manager sal
FORD,3100,JONES,2975
SCOTT,3100,JONES,2975


### Scenario 4: Prepared Statements

Get salary for an employee with a prepared statement

Description:
Get salary for an employee with a prepared statement that declares a fixed query with typed parameters to prevent SQL injection


#### Step 1: 
Prepare a statement employee_salary with a parameterized query:

`prepare stmt_employee_salary(int) as select ename, sal from tbl_employees where empno=$1`

A prepared statement declares parameterized SQL. The following will display the name and salary for a provided employee number (as integer).

> Note:
>  
> The following is a workaround for sql magic and `$1` reference for the parameterized value. This is not needed in any other context. It's just a workaround. When called, the function will create the prepared statement for the %sql connection.

In [11]:
%%sql 
create or replace function fn_tmp_create_stmts() 
returns timestamptz as $DO$
begin
  execute format('prepare stmt_employee_salary(int) as select ename, sal from tbl_employees where empno=$1');
return clock_timestamp(); 
end; 
$DO$ language plpgsql;


 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

To create the prepared statement using the workaround for sql magic, call the function.

In [12]:
a = %sql select fn_tmp_create_stmts()

print (a)

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
+----------------------------------+
|       fn_tmp_create_stmts        |
+----------------------------------+
| 2023-04-20 16:25:20.602389-07:00 |
+----------------------------------+


Verify that the prepared statement exists. There should be four results.

In [13]:
%%sql 
select name from pg_prepared_statements where 1=1 order by name;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
4 rows affected.


name
stmt_employee_salary
stmt_util_metrics_snap_reset
stmt_util_metrics_snap_table
stmt_util_metrics_snap_tablet


Step 2:
Use the EXECUTE statement to execute a previously prepared statement. The following SQL executes the prepared statement with an employee ID 7900.

Once prepared, the statement can be called by providing only the parameter value. This displays the name and salary of employee number 7900.

In [14]:
%%sql

execute stmt_employee_salary(7900);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


ename,sal
JAMES,1050


Step 3: Execute the same prepared statement another employee ID 7902

Without writing the SQL again, this displays the name and salary of employee number 7902

In [15]:
%%sql

execute stmt_employee_salary(7902);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


ename,sal
FORD,3100


A prepared statement stays in the session and can be deallocated when not used anymore. The following frees the memory used by this statement:

In [16]:
%%sql
deallocate stmt_employee_salary;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

Now, check to see that there are now only be 3 prepared statements instead of 4.

In [17]:
%%sql 
select name from pg_prepared_statements where 1=1 order by name;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
3 rows affected.


name
stmt_util_metrics_snap_reset
stmt_util_metrics_snap_table
stmt_util_metrics_snap_tablet


### Scenario 5: Indexes

Create and analyze index on the fly

Description: 
Create a new table and a specific index to avoid table scan and sorts


#### Step 1: Create a new demo table with randomly generated rows

`GENERATE_SERIES()` is a built-in function can generate rows. The following uses it to create a table with 42 rows and a random value from 1 to 10

In [18]:
%%sql 

drop table if exists tbl_demo;

create table if not exists tbl_demo as select generate_series(1,65535) num, round(10*random()) val;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.
65535 rows affected.


[]

#### Step 2: Create the index “demo_val” on demo table

With the goal to query, for a value, with numbers ordered, in the most efficient way, the following creates an index on “val” (hashed for distribution) and “num” in ascending order. You can learn more about hashing and tablets in the next notebook.

In [19]:
%%sql

create index idx_demo_val on tbl_demo (val hash, num asc);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

Describe the index and the table. YugabyteDB uses a LSM-Tree index, not BTREE.

In [20]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}


cd $YB_PATH
./bin/ysqlsh  -d ${DB_NAME} -c "\d+ idx_demo_val" -c "\d+ tbl_demo"


                      Index "public.idx_demo_val"
 Column |       Type       | Key? | Definition | Storage | Stats target 
--------+------------------+------+------------+---------+--------------
 val    | double precision | yes  | val        | plain   | 
 num    | integer          | yes  | num        | plain   | 
lsm, for table "public.tbl_demo"

                                      Table "public.tbl_demo"
 Column |       Type       | Collation | Nullable | Default | Storage | Stats target | Description 
--------+------------------+-----------+----------+---------+---------+--------------+-------------
 num    | integer          |           |          |         | plain   |              | 
 val    | double precision |           |          |         | plain   |              | 
Indexes:
    "idx_demo_val" lsm (val HASH, num ASC)



#### Step 3: Gather optimizer statistics on table demo

The query planner chooses the best access path when provided with statistics about the data stored in the table. The following gathers those statistics.

In [21]:
%%sql 

analyze tbl_demo;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

#### Step 4: Query the Top-3 numbers for a specific value

The following displays the Top-3 numbers for the value 5

In [22]:
%%sql

select * 
from tbl_demo 
where 1=1 
and val=5 
order by num asc 
fetch first 3 rows only;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
3 rows affected.


num,val
7,5.0
10,5.0
23,5.0


#### Step 5: Verify that index is leading to faster query execution using explain analyze

When defining an index for a specific access pattern, it is good that the developer verifies that the index is used. The following shows that an “Index Only Scan” was used, without the need for an additional “Sort” operation

In [23]:
%%sql

explain analyze select * from tbl_demo where val=5 order by num fetch first 3 rows only;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
7 rows affected.


QUERY PLAN
Limit (cost=0.00..0.34 rows=3 width=12) (actual time=0.741..0.744 rows=3 loops=1)
-> Index Only Scan using idx_demo_val on tbl_demo (cost=0.00..741.47 rows=6555 width=12) (actual time=0.740..0.742 rows=3 loops=1)
Index Cond: (val = '5'::double precision)
Heap Fetches: 0
Planning Time: 0.062 ms
Execution Time: 0.776 ms
Peak Memory Usage: 8 kB


#### Step 6: Clean up the table for this exercise.

To leave the database in the same state as before this exercise, the following removes the demo table created before

In [24]:
%%sql

drop table if exists tbl_demo;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

## q2 | Built-in Functions
Learn powerful functions for performing complex database operations with ease

### Scenario 1: Window Functions
Compare employees hiring time interval by department using LAG function

LAG is a window function that provides access to the row before the current one. The following SQL statement uses WINDOW to define groups of employees by department, in order of their hiring date. LAG is used to access the previous row in this group, to compare the hiring date interval between two employees. FORMAT builds a text from column values, and COALESCE handles the first hire for which there is no previous row in the group. Without those window functions, this query would have required reading the same table two times.


In [25]:
%%sql

select
dname,ename,job,
coalesce (
  'hired '||to_char(hiredate -
    lag(hiredate) over (per_dept_hiredate),'999')||' days after '||
    lag(ename) over (per_dept_hiredate),
    format('(1st hire in %L)',dname)
) as "last hire in dept"
from tbl_employees join tbl_departments using(deptno)
window per_dept_hiredate
  as (partition by dname order by hiredate)
order by dname,hiredate;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
14 rows affected.


dname,ename,job,last hire in dept
ACCOUNTING,CLARK,MANAGER,(1st hire in 'ACCOUNTING')
ACCOUNTING,KING,PRESIDENT,hired 161 days after CLARK
ACCOUNTING,MILLER,CLERK,hired 67 days after KING
RESEARCH,SMITH,CLERK,(1st hire in 'RESEARCH')
RESEARCH,JONES,MANAGER,hired 106 days after SMITH
RESEARCH,FORD,DEVELOPER,hired 245 days after JONES
RESEARCH,SCOTT,ANALYST,hired 371 days after FORD
RESEARCH,ADAMS,CLERK,hired 34 days after SCOTT
SALES,ALLEN,SALESMAN,(1st hire in 'SALES')
SALES,WARD,SALESMAN,hired 2 days after ALLEN


### Scenario 2. Regexp Matching

List all employees with `@gmail` or `.org` in their email addresses

Description: `REGEXP` performs a pattern match of a string expression. The following lists employees with an e-mail ending in `.org` or a domain starting with `gmail`.

In [26]:
%%sql

select * 
from tbl_employees
where 1=1
and email ~ any ( ARRAY[ '@.*\.org$' , '@gmail\.' ] );

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
3 rows affected.


empno,ename,job,mgr,hiredate,sal,comm,deptno,email,other_info
7876,ADAMS,CLERK,7788,1983-01-12,1200,None,20,ADAMS@acme.org,None
7566,JONES,MANAGER,7839,1981-04-02,2975,None,20,JONES@gmail.com,None
7900,JAMES,CLERK,7698,1981-12-03,1050,None,30,JAMES@acme.org,None


### Scenario 3. Arithmetic Date Intervals

The interval data type allows you to store and manipulate a period of time in years, months, days. 

The following example compares overlapping evaluation periods. A WITH clause defines the evaluation period length depending on the job.

Find employees with overlapping evaluation periods.

In [27]:
%%sql

with employees_evaluation_period  as (
 select ename,deptno,hiredate,
 hiredate + case when job in ('MANAGER','PRESIDENT')
 then interval '3 month' else interval '4 weeks'
 end evaluation_end 
 from tbl_employees)
select * from employees_evaluation_period e1
 join  employees_evaluation_period e2
 on (e1.ename>e2.ename) and (e1.deptno=e2.deptno)
where (e1.hiredate,e1.evaluation_end)
 overlaps (e2.hiredate,e2.evaluation_end);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
3 rows affected.


ename,deptno,hiredate,evaluation_end,ename_1,deptno_1,hiredate_1,evaluation_end_1
MILLER,10,1982-01-23,1982-02-20 00:00:00,KING,10,1981-11-17,1982-02-17 00:00:00
TURNER,30,1981-09-08,1981-10-06 00:00:00,MARTIN,30,1981-09-28,1981-10-26 00:00:00
WARD,30,1981-02-22,1981-03-22 00:00:00,ALLEN,30,1981-02-20,1981-03-20 00:00:00


### Scenario 4. CROSSTABVIEW

`CROSSTABVIEW` is a client meta-command to display rows as columns. The following sums the salaries across jobs and departments and displays them as a cross-table

Display total salary per job and department as a cross-table using bash

In [28]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}

#SQL_CROSSTABVIEW="\crosstabview"

cd $YB_PATH
echo 'select job, dname, sum(sal) from tbl_employees join tbl_departments using(deptno) group by dname, job  \crosstabview' | ./bin/ysqlsh  -d ${DB_NAME}


    job    | ACCOUNTING | SALES | RESEARCH 
-----------+------------+-------+----------
 PRESIDENT |       5100 |       |         
 CLERK     |       1400 |  1050 |     2100
 SALESMAN  |            |  6000 |         
 MANAGER   |       2450 |  2850 |     2975
 ANALYST   |            |       |     3100
 DEVELOPER |            |       |     3100
(6 rows)



### Scenario 5: NTILE Function

Split e-mails in 3 groups and format them

Description: In order to send e-mails to all employees in different batches, you will split them into 3 groups using the NTILE function, and format them with the FORMAT function and aggregate them in a comma-separated list with the STRING_AGG function.

This is done using the following Common Table Expression, CTE:

In [29]:
%%sql

with groups as (
 select ntile(3) over (order by empno) group_num
 ,* 
 from tbl_employees
)
select string_agg(format('<%s> %s',ename,email),', ') 
from groups group by group_num;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
3 rows affected.


string_agg
"<ADAMS> ADAMS@acme.org, <JAMES> JAMES@acme.org, <FORD> FORD@acme.com, <MILLER> MILLER@acme.com"
"<BLAKE> BLAKE@hotmail.com, <CLARK> CLARK@acme.com, <SCOTT> SCOTT@acme.com, <KING> KING@aol.com, <TURNER> TURNER@acme.com"
"<SMITH> SMITH@acme.com, <ALLEN> ALLEN@acme.com, <WARD> WARD@compuserve.com, <JONES> JONES@gmail.com, <MARTIN> MARTIN@acme.com"


## q3 | Advanced Features
Expand your YSQL skills by completing the following 5 scenarios

### Scenario 1:  GIN Index on Document
List employees that know SQL

Description: The skills are stored in the semi-structured JSON document. We can query them with @>, ?, ?& and ?| operators. And, for best performance, index them.

#### Step 1: Create GIN index on JSON document

GIN indexes can provide fast access to elements inside a JSON document. The following creates an index on the ‘skills’ attributes within ‘other_info’ JSON column

In [30]:
%%sql 

create index idx_emp_skills_ybgin on tbl_employees using gin((other_info->'skills'));

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

Describe the table and the index.

In [31]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}


cd $YB_PATH
./bin/ysqlsh  -d ${DB_NAME} -c "\d+ idx_emp_skills_ybgin" -c "\d+ tbl_employees"


                       Index "public.idx_emp_skills_ybgin"
 Column | Type | Key? |           Definition           | Storage  | Stats target 
--------+------+------+--------------------------------+----------+--------------
 expr   | text | yes  | (other_info -> 'skills'::text) | extended | 
ybgin, for table "public.tbl_employees"

                                              Table "public.tbl_employees"
   Column   |  Type   | Collation | Nullable |             Default              | Storage  | Stats target | Description 
------------+---------+-----------+----------+----------------------------------+----------+--------------+-------------
 empno      | integer |           | not null | generated by default as identity | plain    |              | 
 ename      | text    |           | not null |                                  | extended |              | 
 job        | text    |           |          |                                  | extended |              | 
 mgr        | integer |

#### Step 2: Query the JSON attribute list

SQL queries can navigate into the JSON document with -> and check if an array contains a value with @>. The following searches the employees with the “SQL” skill

In [32]:
%%sql 

select * from tbl_employees where other_info->'skills' @> '"SQL"' ;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


empno,ename,job,mgr,hiredate,sal,comm,deptno,email,other_info
7902,FORD,DEVELOPER,7566,1981-12-03,3100,None,20,FORD@acme.com,"{'skills': ['YSQL', 'YCQL', 'SQL', 'CQL']}"
7782,CLARK,MANAGER,7839,1981-06-09,2450,None,10,CLARK@acme.com,"{'skills': ['C', 'C++', 'SQL']}"


Step 3: Use the Explain plan to verify that index usage

Thanks to the GIN index, this search doesn’t need to read all documents. The following shows the execution plan being an indexed access path.

In [33]:
%%sql

explain select * from tbl_employees where other_info->'skills' @> '"SQL"' ;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


QUERY PLAN
Index Scan using idx_emp_skills_ybgin on tbl_employees (cost=4.00..12.01 rows=1000 width=152)
"Index Cond: ((other_info -> 'skills'::text) @> '""SQL""'::jsonb)"


### Scenario 2: Text Search
Build a search index on department descriptions

Description: 
SQL queries can search in the text for some words using the `to_tsvector()` function to extract a list of words that can be compared. We will find all department descriptions with the words 'responsible' and 'services' in it

#### Step 1: Create text search index on the description column
General Inverted Indexes, known as GIN indexes, can provide fast access to words inside a text. The following creates an index for the simple-grammar vector of words extracted from the department description.

In [34]:
%%sql

create index idx_dept_description_ybgin on tbl_departments using gin (( to_tsvector('simple',description) ));

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

Describe the index and the table.

In [35]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}


cd $YB_PATH
./bin/ysqlsh  -d ${DB_NAME} -c "\d+ idx_dept_description_ybgin" -c "\d+ tbl_departments"


                              Index "public.idx_dept_description_ybgin"
   Column    | Type | Key? |                  Definition                   | Storage  | Stats target 
-------------+------+------+-----------------------------------------------+----------+--------------
 to_tsvector | text | yes  | to_tsvector('simple'::regconfig, description) | extended | 
ybgin, for table "public.tbl_departments"

                                 Table "public.tbl_departments"
   Column    |  Type   | Collation | Nullable | Default | Storage  | Stats target | Description 
-------------+---------+-----------+----------+---------+----------+--------------+-------------
 deptno      | integer |           | not null |         | plain    |              | 
 dname       | text    |           |          |         | extended |              | 
 loc         | text    |           |          |         | extended |              | 
 description | text    |           |          |         | extended |           

#### Step 2: Query on description for matching words

The following compares the simple-grammar vector of words extracted from the department description with a word search pattern to find the departments which contain ‘responsible’ and ‘services’ in their description.

In [36]:
%%sql

select * from tbl_departments where to_tsvector('simple',description) @@ to_tsquery('simple','responsible & services');

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


deptno,dname,loc,description
30,SALES,CHICAGO,division of a business that is responsible for selling products or services


#### Step 3: Explain plan to verify that index is used

Thanks to the GIN index, this search doesn’t need to read all rows and text. The following shows the execution plan being an indexed access path

In [37]:
%%sql

explain select * from tbl_departments where to_tsvector('simple',description) @@ to_tsquery('simple','responsible & services');

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


QUERY PLAN
Index Scan using idx_dept_description_ybgin on tbl_departments (cost=8.00..16.01 rows=1000 width=100)
"Index Cond: (to_tsvector('simple'::regconfig, description) @@ '''responsible'' & ''services'''::tsquery)"


### Scenario 3. Stored Procedures
Transfer commission from one employee to another


Description: 
A stored procedure can encapsulate a procedural logic into an atomic operation. We will create one in PL/pgSQL, named "commission_transfer", that transfers commission “amount” from “empno1” to “empno2”


#### Step 1: Create the procedure for the commission transfer between employees

The procedure has two SQL operations: 
- decrease from `empno1`
- add to `empno2`


In addition, the procedure must check for an error and raise a custom exception if `empno1` does not have the amount to be transferred

In [38]:
%%sql

create or replace procedure commission_transfer(empno1 int, empno2 int, amount int) 
as $$
begin
  update tbl_employees set comm=comm-commission_transfer.amount
    where tbl_employees.empno=commission_transfer.empno1 and comm>commission_transfer.amount;
  if not found then 
    raise exception 'Cannot transfer % from %',amount,empno1;
  end if;
  update tbl_employees set comm=comm+commission_transfer.amount
    where tbl_employees.empno=commission_transfer.empno2;
  if not found then 
    raise exception 'Cannot transfer from %',empno2; 
  end if;
end;
$$ language plpgsql;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

#### Step 2: Call the procedure with employee IDs and the amount to be transferred

Once defined, the stored procedure is called with values for all parameters. This transfers 100 from tbl_employeesloyee 7521 to 7654

In [39]:
%%sql
select * from  tbl_employees where empno in (7521,7654);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


empno,ename,job,mgr,hiredate,sal,comm,deptno,email,other_info
7521,WARD,SALESMAN,7698,1981-02-22,1350,500,30,WARD@compuserve.com,None
7654,MARTIN,SALESMAN,7698,1981-09-28,1350,1400,30,MARTIN@acme.com,None


In [40]:
%%sql

call commission_transfer(7521,7654,100);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

In [41]:
%%sql
select * from  tbl_employees where empno in (7521,7654);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


empno,ename,job,mgr,hiredate,sal,comm,deptno,email,other_info
7521,WARD,SALESMAN,7698,1981-02-22,1350,400,30,WARD@compuserve.com,None
7654,MARTIN,SALESMAN,7698,1981-09-28,1350,1500,30,MARTIN@acme.com,None


#### Step 3: List all employees who have received commission

The following displays all employees having a commission, to verify that 100 have been transferred

In [42]:
%%sql

select * from tbl_employees where comm is not null;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
4 rows affected.


empno,ename,job,mgr,hiredate,sal,comm,deptno,email,other_info
7521,WARD,SALESMAN,7698,1981-02-22,1350,400,30,WARD@compuserve.com,None
7654,MARTIN,SALESMAN,7698,1981-09-28,1350,1500,30,MARTIN@acme.com,None
7499,ALLEN,SALESMAN,7698,1981-02-20,1700,300,30,ALLEN@acme.com,None
7844,TURNER,SALESMAN,7698,1981-09-08,1600,0,30,TURNER@acme.com,None


#### Step 4: Call the procedure with employee IDs and amount that is not allowed to invoke error handling

The following attempts to transfer 1000000, more than what 7521 has. It raises the “Cannot transfer” error defined in the procedure and automatically reverts all intermediate changes to return to a consistent state.


In [43]:
%%sql

call commission_transfer(7521,7654,1000000);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
(psycopg2.errors.RaiseException) Cannot transfer 1000000 from 7521
CONTEXT:  PL/pgSQL function commission_transfer(integer,integer,integer) line 6 at RAISE

[SQL: call commission_transfer(7521,7654,1000000);]
(Background on this error at: https://sqlalche.me/e/20/2j85)


> Note: By design, the above raises the exception.

### Scenario 4. Triggers
Record the last update time of each row automatically

Description
We will add a column to hold the last update time, and declare a trigger to update it automatically, on the departments table


#### Step 1: Add a column to store the last update time

The structure of a SQL table can evolve. With the goal of recording the last update, the following adds a “last_update” column to the department table.

In [44]:
%%sql

alter table tbl_departments add last_update timestamptz;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

#### Step 2: Create a function

Add a function `fn_dept_last_update` to set the last update time. The following SQL query uses the built-in function transaction_timestamp(), which returns the current date and time at the start of the current transaction

A stored function declares some procedural logic that returns a value. The following returns the “new” state for a trigger after setting the “last_update” to the current time.

In [45]:
%%sql

create or replace function fn_dept_last_update() returns trigger as $$
begin
  new.last_update:=transaction_timestamp();
  return new;
end;
$$ language plpgsql;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

#### Step 3: Create a trigger

Create a trigger `trg_dept_last_update` to call the function `fn_dept_last_update()` on each table update.

The previous function can be called automatically. The following trigger executes it on each row update for the departments table.

In [46]:
%%sql

create trigger trg_dept_last_update
before update on tbl_departments
for each row
execute procedure fn_dept_last_update();

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

Describe the table.

In [47]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}


cd $YB_PATH
./bin/ysqlsh  -d ${DB_NAME} -c "\d+ tbl_departments"


                                         Table "public.tbl_departments"
   Column    |           Type           | Collation | Nullable | Default | Storage  | Stats target | Description 
-------------+--------------------------+-----------+----------+---------+----------+--------------+-------------
 deptno      | integer                  |           | not null |         | plain    |              | 
 dname       | text                     |           |          |         | extended |              | 
 loc         | text                     |           |          |         | extended |              | 
 description | text                     |           |          |         | extended |              | 
 last_update | timestamp with time zone |           |          |         | plain    |              | 
Indexes:
    "pk_dept" PRIMARY KEY, lsm (deptno ASC)
    "idx_dept_description_ybgin" ybgin (to_tsvector('simple'::regconfig, description))
Triggers:
    trg_dept_last_update BEFORE UPDATE O

#### Step 4: Display the current state of the table

In order to verify the automatic logging of the last update time, the following displays the current state of departments before any update

In [48]:
%%sql

select '' _
  , deptno
  , dname
  , loc
  , last_update 
from tbl_departments;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
4 rows affected.


_,deptno,dname,loc,last_update
,10,ACCOUNTING,NEW YORK,None
,20,RESEARCH,DALLAS,None
,30,SALES,CHICAGO,None
,40,OPERATIONS,BOSTON,None


#### Step 5: Update multiple rows in one transaction in the dept table

Multiple updates can be declared in a single atomic transaction with BEGIN TRANSACTION and COMMIT. The following updates the location of departments 30 and 40 with a 3 seconds interval in between.


In [49]:
%%sql
-- begin transaction;-- commit;
update tbl_departments set loc='SUNNYVALE' where deptno=30;
select pg_sleep(3);
update tbl_departments set loc='SUNNYVALE' where deptno=40;


 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
1 rows affected.
1 rows affected.


[]

#### Step 6: Display the new state of the table to see the last_update column being updated

The following shows that in addition of the change location, the last update timestamp has been automatically set. Even if the updates were done at 3 seconds interval, they show the same update time because all was run in an atomic transaction


In [50]:
%%sql
select '' _
  , deptno
  , dname
  , loc
  , last_update 
from tbl_departments;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
4 rows affected.


_,deptno,dname,loc,last_update
,10,ACCOUNTING,NEW YORK,None
,20,RESEARCH,DALLAS,None
,30,SALES,SUNNYVALE,2023-04-20 16:32:29.530197-07:00
,40,OPERATIONS,SUNNYVALE,2023-04-20 16:32:32.544431-07:00


### Scenario 5. Materialized Views
View
Pre-compute analytics for reporting, with a materialized view

Description
In order to get fast on-demand reports, we create a materialized view to store pre-joined and pre-aggregated data. This view will store the total salary per department, the number of employees, and the list of jobs in the department.


#### Step 1: Create the materialized view

In [51]:
%%sql

create materialized view mvw_report_sal_per_dept as
select 
    deptno,
    dname,
    sum(sal) sal_per_dept,
    count(*) num_of_employees,
    string_agg(distinct job,', ') distinct_jobs
from tbl_departments join tbl_employees using(deptno)
group by deptno,dname
order by deptno;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
3 rows affected.


[]

Describe the view.

In [52]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}


cd $YB_PATH
./bin/ysqlsh  -d ${DB_NAME} -c "\d+ mvw_report_sal_per_dept"

                         Materialized view "public.mvw_report_sal_per_dept"
      Column      |  Type   | Collation | Nullable | Default | Storage  | Stats target | Description 
------------------+---------+-----------+----------+---------+----------+--------------+-------------
 deptno           | integer |           |          |         | plain    |              | 
 dname            | text    |           |          |         | extended |              | 
 sal_per_dept     | bigint  |           |          |         | plain    |              | 
 num_of_employees | bigint  |           |          |         | plain    |              | 
 distinct_jobs    | text    |           |          |         | extended |              | 
View definition:
 SELECT tbl_departments.deptno,
    tbl_departments.dname,
    sum(tbl_employees.sal) AS sal_per_dept,
    count(*) AS num_of_employees,
    string_agg(DISTINCT tbl_employees.job, ', '::text) AS distinct_jobs
   FROM tbl_departments
     JOIN tbl_employ

#### Step 2: Create an index for the materialized view

Indexes can be created on it. This one allows fast queries on a range of total salary

In [53]:
%%sql
drop index if exists idx_mvw_report_sal_per_dept_sal;
create index if not exists idx_mvw_report_sal_per_dept_sal on mvw_report_sal_per_dept(sal_per_dept desc);

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.
Done.


[]

Describe the index for the materialized view.

In [54]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}


cd $YB_PATH
./bin/ysqlsh  -d ${DB_NAME} -c "\d+ idx_mvw_report_sal_per_dept_sal"

            Index "public.idx_mvw_report_sal_per_dept_sal"
    Column    |  Type  | Key? |  Definition  | Storage | Stats target 
--------------+--------+------+--------------+---------+--------------
 sal_per_dept | bigint | yes  | sal_per_dept | plain   | 
lsm, for table "public.mvw_report_sal_per_dept"



#### Step 3: Refresh

A refresh can be scheduled on a daily basis to re-compute it in the background with a simple command.

In [55]:
%%sql

refresh materialized view mvw_report_sal_per_dept;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
Done.


[]

#### Step 4: Query

Querying the materialized view will show the data consistent as of the last refresh. This one lists the departments with a total salary lower than 10000

In [56]:
%%sql

select *
from mvw_report_sal_per_dept
where 1=1
and sal_per_dept<=10000
order by sal_per_dept;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


deptno,dname,sal_per_dept,num_of_employees,distinct_jobs
10,ACCOUNTING,8950,3,"CLERK, MANAGER, PRESIDENT"
30,SALES,9900,6,"CLERK, MANAGER, SALESMAN"


#### Step 5: View the query plan

The execution plan shows that no additional join or group by are needed when querying this materialized view:

In [57]:
%%sql
explain analyze
select *
from mvw_report_sal_per_dept
where 1=1
and sal_per_dept<=10000
order by sal_per_dept;

 * postgresql://yugabyte@127.0.0.1:5433/db_ybu
5 rows affected.


QUERY PLAN
Index Scan Backward using idx_mvw_report_sal_per_dept_sal on mvw_report_sal_per_dept (cost=0.00..5.33 rows=10 width=84) (actual time=1.358..1.362 rows=2 loops=1)
Index Cond: (sal_per_dept <= 10000)
Planning Time: 0.060 ms
Execution Time: 1.390 ms
Peak Memory Usage: 0 kB


---
# 🌟🌟 Well done! 
In this notebook, you completed the following:
- YSQL Development
  - Basic of DDL and DML
  - Built-in Functions
  - Advanced Language features


## 😊 Next up!
Continue your learning by opening the next notebook, `03_Demystifying_table_sharding_tablets_and_data_distribution.ipynb`. 

Or, to open the notebook from GitPod, run the following:

In [ ]:
%%bash
gp open '03_Demystifying_table_sharding_tablets_and_data_distribution.ipynb'